In [131]:
import json
import requests
import pandas as pd

In [142]:
#simplest way to query any API is through a get request (url in quotations is same as what you paste in browser)
crossref = requests.get(f"https://api.crossref.org/works?filter=doi:10.1145/3472538.3472567,doi:10.1145/3472538.3472569,doi:10.1145/3469013.3469014&select=DOI,title,funder&mailto=paula.schirrmacher@york.ac.uk&rows=3")

crossref.json()

{'status': 'ok',
 'message-type': 'work-list',
 'message-version': '1.0.0',
 'message': {'facets': {},
  'total-results': 3,
  'items': [{'indexed': {'date-parts': [[2023, 1, 15]],
     'date-time': '2023-01-15T06:04:39Z',
     'timestamp': 1673762679024},
    'publisher-location': 'New York, NY, USA',
    'reference-count': 26,
    'publisher': 'ACM',
    'content-domain': {'domain': ['dl.acm.org'],
     'crossmark-restriction': True},
    'published-print': {'date-parts': [[2021, 7, 28]]},
    'DOI': '10.1145/3469013.3469014',
    'type': 'proceedings-article',
    'created': {'date-parts': [[2021, 7, 12]],
     'date-time': '2021-07-12T00:39:08Z',
     'timestamp': 1626050348000},
    'update-policy': 'http://dx.doi.org/10.1145/crossmark-policy',
    'source': 'Crossref',
    'is-referenced-by-count': 0,
    'title': ['An archive-exploration system for the Hunting Songs of the Lakeland Fell Packs'],
    'prefix': '10.1145',
    'author': [{'given': 'Mary',
      'family': 'Emmett',


In [148]:
#general query crossref API, step-by-step
API_endpoint= "https://api.crossref.org/works?"
email= "YOUREMAIL@something.com" #polite query includes contact details - leads to faster results
items_per_page="3"
export="DOI,title,funder" #or any other field, e.g. published or published-online...
query="doi:10.1145/3472538.3472567,doi:10.1145/3472538.3472569,doi:10.1145/3469013.3469014" #cumbersome string with repeated "DOI:"

crossref=requests.get(f"{API_endpoint}filter={query}&select={export}&mailto={email}&rows={items_per_page}")
crossref.json()

{'status': 'ok',
 'message-type': 'work-list',
 'message-version': '1.0.0',
 'message': {'facets': {},
  'total-results': 3,
  'items': [{'DOI': '10.1145/3469013.3469014',
    'title': ['An archive-exploration system for the Hunting Songs of the Lakeland Fell Packs'],
    'published': {'date-parts': [[2021, 7, 28]]},
    'published-online': {'date-parts': [[2021, 7, 28]]}},
   {'DOI': '10.1145/3472538.3472569',
    'title': ['“Slow down and look”: Desirable aspects of failure in video games, from the perspective of players.'],
    'published': {'date-parts': [[2021, 8, 3]]},
    'published-online': {'date-parts': [[2021, 10, 21]]}},
   {'DOI': '10.1145/3472538.3472567',
    'title': ['Grounded Theory of Accessible Game Development'],
    'funder': [{'name': 'UK Engineering and Physical Sciences Research Council',
      'award': ['EP/S022325/1']}],
    'published': {'date-parts': [[2021, 8, 3]]},
    'published-online': {'date-parts': [[2021, 10, 21]]}}],
  'items-per-page': 3,
  'query

In [151]:
#####################################################
##### use crossref API by making changes below ######
#####################################################
API_endpoint= "https://api.crossref.org/works?"
email= "paula.schirrmacher@york.ac.uk" #polite
items_per_page="3" #limiting number of outputs is particularly important for open search queries (e.g. query.author="Jane Smith")
export="DOI,title,funder" #or any other field e.g. published
query_csv="DOIs_test.csv" #csv file with list of DOIs (in columns or rows)
#####################################################
##### make your changes in above block ##############
#####################################################

query_DOIs = pd.read_csv(query_csv, header=None)
query_DOIs1 = query_DOIs.astype(str).values.flatten().tolist()
DOIs=[]
for idx, i in enumerate(query_DOIs1):
        DOIs.append(f"doi:{i}")
        query=','.join(DOIs)
query

crossref=requests.get(f"{API_endpoint}filter={query}&select={export}&mailto={email}&rows={items_per_page}")
crossref.json()

{'status': 'ok',
 'message-type': 'work-list',
 'message-version': '1.0.0',
 'message': {'facets': {},
  'total-results': 3,
  'items': [{'DOI': '10.1145/3469013.3469014',
    'title': ['An archive-exploration system for the Hunting Songs of the Lakeland Fell Packs']},
   {'DOI': '10.1145/3472538.3472569',
    'title': ['“Slow down and look”: Desirable aspects of failure in video games, from the perspective of players.']},
   {'DOI': '10.1145/3472538.3472567',
    'title': ['Grounded Theory of Accessible Game Development'],
    'funder': [{'name': 'UK Engineering and Physical Sciences Research Council',
      'award': ['EP/S022325/1']}]}],
  'items-per-page': 3,
  'query': {'start-index': 0, 'search-terms': None}}}

In [152]:
#convert json to table format
#problem: not all fields have 'funder' info
#'explode' method is used to unlist funder data and 'apply' adds zero, wherever data unavailable
#change 'funder' to any other field that is occasionally empty

subset=crossref.json()['message']['items']
df1 = pd.DataFrame(subset).explode("funder")
df1 = pd.concat([df1, df1.pop("funder").apply(pd.Series)], axis=1).drop(
    columns=0
)

pretty = pd.DataFrame(df1)
pretty

,DOI,title,name,award
0,10.1145/3469013.3469014,[An archive-exploration system for the Hunting...,NaN,NaN
1,10.1145/3472538.3472569,[“Slow down and look”: Desirable aspects of fa...,NaN,NaN
2,10.1145/3472538.3472567,[Grounded Theory of Accessible Game Development],UK Engineering and Physical Sciences Research ...,[EP/S022325/1]


In [141]:
#export to csv
df1.to_csv('test_crossref_export.csv')